In [7]:
import serial
import time
from threading import Thread
import binascii

class NeuroPy:
    def __init__(self, port="COM4", baudRate=57600,
                 on_attention=None,
                 on_meditation=None,
                 on_raw=None,
                 on_poor_signal=None,
                 on_blink=None):

        self.port = port
        self.baudRate = baudRate
        self.serial = None
        self.running = False
        self.thread = None

        # Brainwave values
        self.attention = 0
        self.meditation = 0
        self.rawValue = 0
        self.poorSignal = 0
        self.blinkStrength = 0

        # Callbacks
        self.callbacks = {
            "attention": on_attention,
            "meditation": on_meditation,
            "rawValue": on_raw,
            "poorSignal": on_poor_signal,
            "blinkStrength": on_blink
        }

    def __fireEvent(self, name, value):
        if self.callbacks.get(name):
            try:
                self.callbacks[name](value)
            except Exception as e:
                print(f"Callback error for {name}: {e}")

    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudRate)
            self.serial.flushInput()
            print(f"Connected to {self.port}")
            return True
        except serial.SerialException as e:
            print(f"Error connecting to {self.port}: {e}")
            return False

    def start(self):
        if not self.serial:
            if not self.connect():
                return
        self.running = True
        self.thread = Thread(target=self._read_loop)
        self.thread.start()

    def stop(self):
        self.running = False
        if self.thread:
            self.thread.join()
        if self.serial:
            self.serial.close()

    def _read_loop(self):
        while self.running:
            try:
                if binascii.hexlify(self.serial.read(1)) != b'aa':
                    continue
                if binascii.hexlify(self.serial.read(1)) != b'aa':
                    continue

                payload_length = int.from_bytes(self.serial.read(1), 'big')
                payload = self.serial.read(payload_length)
                checksum = int.from_bytes(self.serial.read(1), 'big')
                calc_checksum = (~sum(payload) & 0xFF)

                if checksum != calc_checksum:
                    continue

                i = 0
                while i < len(payload):
                    code = payload[i]
                    i += 1
                    if code == 0x02:
                        self.poorSignal = payload[i]
                        self.__fireEvent("poorSignal", self.poorSignal)
                        i += 1
                    elif code == 0x04:
                        self.attention = payload[i]
                        self.__fireEvent("attention", self.attention)
                        i += 1
                    elif code == 0x05:
                        self.meditation = payload[i]
                        self.__fireEvent("meditation", self.meditation)
                        i += 1
                    elif code == 0x16:
                        self.blinkStrength = payload[i]
                        self.__fireEvent("blinkStrength", self.blinkStrength)
                        i += 1
                    elif code == 0x80:
                        i += 1  # Skip length byte
                        high_byte = payload[i]
                        low_byte = payload[i + 1]
                        raw = (high_byte << 8) | low_byte
                        if raw > 32768:
                            raw -= 65536
                        self.rawValue = raw
                        self.__fireEvent("rawValue", self.rawValue)
                        i += 2
                    else:
                        # Skip unknown codes
                        if i < len(payload):
                            i += 1
            except Exception as e:
                print(f"Read loop error: {e}")

# === Example Usage ===
# if __name__ == "__main__":
#     def on_attention(value):
#         print(f"[Attention] {value}")

#     def on_meditation(value):
#         print(f"[Meditation] {value}")

#     def on_raw(value):
#         print(f"[Raw EEG] {value}")

#     neuro = NeuroPy(
#         port="COM4",  # Replace with your port
#         on_attention=on_attention,
#         on_meditation=on_meditation,
#         on_raw=on_raw
#     )

#     try:
#         neuro.start()
#         while True:
#             time.sleep(1)
#     except KeyboardInterrupt:
#         neuro.stop()
#         print("Disconnected.")


import csv
import os
from datetime import datetime
import time

if __name__ == "__main__":
    csv_file = f"notes_{time.strftime('%y%m%d_%H%M%S')}.csv"
    # Write header if file does not exist
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "type", "value"])

    def log_to_csv(label, value):
        with open(csv_file, "a", newline="") as f:
            writer = csv.writer(f)
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]  # up to milliseconds
            writer.writerow([timestamp, label, value])            

    def on_attention(value):
        print(f"[Attention] {value}")
        log_to_csv("Attention", value)

    def on_meditation(value):
        print(f"[Meditation] {value}")
        log_to_csv("Meditation", value)

    def on_raw(value):
        print(f"[Raw EEG] {value}")
        log_to_csv("Raw EEG", value)

    neuro = NeuroPy(
        port="COM4",  # Replace with your port
        on_attention=on_attention,
        on_meditation=on_meditation,
        on_raw=on_raw

    )

    try:
        print("Starting NeuroPy...")
        neuro.start()
        print("NeuroPy started. Press Ctrl+C to stop.")
        start_time = time.time()
        while time.time() - start_time < 60:  # Run for 30 seconds
            time.sleep(1)
        neuro.stop()
        print("Stopped after 60 seconds.")
    except KeyboardInterrupt:
        neuro.stop()
        print("Disconnected.")



Starting NeuroPy...
Connected to COM4
NeuroPy started. Press Ctrl+C to stop.
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 29
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 29
[Raw EEG] 29
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 27
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 27
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 28
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 27
[Raw EEG] 28
[Raw EEG] 28
[Raw EEG] 28
